In [1]:
%pip install requests
import requests

import warnings
warnings.filterwarnings('ignore')


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from utils.sessions import *
from utils.laps import *
from utils.weather import *
from utils.positions import *
from utils.combine import *
from utils.previous import *
import logging
import os
from datetime import datetime
import json
import time

# Set up logging
log_dir = 'logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file = os.path.join(log_dir, f'data_processing_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
race_weekends = get_all_race_weekends()
combined_df = pd.DataFrame()
position_history = {}

loop_count = 1
total_weekends = len(race_weekends)
max_retries = 3

for race_weekend in race_weekends:
    for attempt in range(max_retries):
        try:
            logging.info(f'Processing {race_weekend["location"]} {race_weekend["year"]} (Attempt {attempt+1}/{max_retries})')
            print(f'Processing {race_weekend["location"]} {race_weekend["year"]}', end='\r', flush=True)
            
            practice_session_keys = get_weekend_session_keys(race_weekend, 'Practice')
            logging.debug(f'Practice session keys: {practice_session_keys}')
            
            lap_data = []
            for session_key in practice_session_keys:
                for session_attempt in range(max_retries):
                    try:
                        lap_data.append(practice_session_combined_data(session_key))
                        logging.debug(f'Successfully processed session {session_key}')
                        break
                    except Exception as e:
                        logging.error(f'Error processing session {session_key} (Attempt {session_attempt+1}/{max_retries}): {str(e)}')
                        if session_attempt < max_retries - 1:
                            time.sleep(2**session_attempt)  # Exponential backoff
                            continue
                        break

            all_practice_data = combine_all_practices(lap_data)
            practice_statistics = extract_data_from_session(all_practice_data)
            practice_statistics = create_ran_flags(practice_statistics)

            practice_statistics = add_statistic_differentials_per_event(practice_statistics)
            practice_statistics = fill_not_ran_nan(practice_statistics)

            for weather_attempt in range(max_retries):
                try:
                    weather_data = get_weather_by_session_keys(practice_session_keys)
                    practice_statistics = add_weather_data_to_event_practice_statistics(practice_statistics, weather_data)
                    break
                except Exception as e:
                    logging.error(f'Error getting weather data (Attempt {weather_attempt+1}/{max_retries}): {str(e)}')
                    if weather_attempt < max_retries - 1:
                        time.sleep(2**weather_attempt)
                        continue

            quali_positions_df = None
            race_positions_df = None

            for quali_attempt in range(max_retries):
                try:
                    quali_positions = get_weekend_session_keys(race_weekend, 'Qualifying')
                    quali_positions_df = get_end_positions(quali_positions[0])

                    practice_statistics = practice_statistics.merge(
                        quali_positions_df[['driver_number', 'position']], 
                        on='driver_number', 
                        how='left'
                    ).rename(columns={'position': 'quali_position'})
                    break
                except Exception as e:
                    logging.error(f'Error getting qualifying positions (Attempt {quali_attempt+1}/{max_retries}): {str(e)}')
                    if quali_attempt < max_retries - 1:
                        time.sleep(2**quali_attempt)
                        continue
                    break

            for race_attempt in range(max_retries):
                try:
                    race_positions = get_weekend_session_keys(race_weekend, 'Race')
                    race_positions_df = get_end_positions(race_positions[0])

                    practice_statistics = practice_statistics.merge(
                        race_positions_df[['driver_number', 'position']],
                        on='driver_number',
                        how='left'
                    ).rename(columns={'position': 'race_position'})
                    break
                except Exception as e:
                    logging.error(f'Error getting race positions (Attempt {race_attempt+1}/{max_retries}): {str(e)}')
                    if race_attempt < max_retries - 1:
                        time.sleep(2**race_attempt)
                        continue

            # Add previous n events data. -1 Represents all events.
            previous_n_events = [-1, 1, 3, 5, 10]
            for previous_attempt in range(max_retries):
                try:
                    for n_value in previous_n_events:
                        qualifying_events, race_events = retrieve_previous_n_events(position_history, n_value)
                        previous_events = aggregate_previous_n_events(qualifying_events, race_events, n_value)
                        practice_statistics = add_previous_n_events(practice_statistics, previous_events, n_value)
                    break
                except Exception as e:
                    logging.error(f'Error getting previous events data (Attempt {previous_attempt+1}/{max_retries}): {str(e)}')
                    if previous_attempt < max_retries - 1:
                        time.sleep(2**previous_attempt)
                        continue

            # Store position data in history dictionary
            if quali_positions_df is not None or race_positions_df is not None:
                position_history[race_weekend['meeting_key']] = {
                    'year': race_weekend['year'],
                    'qualifying': [] if quali_positions_df is None else [
                        {'driver': int(row['driver_number']), 'position': int(row['position'])} 
                        for _, row in quali_positions_df.iterrows()
                    ],
                    'race': [] if race_positions_df is None else [
                        {'driver': int(row['driver_number']), 'position': int(row['position'])}
                        for _, row in race_positions_df.iterrows()
                    ]
                }

            if combined_df.empty:
                combined_df = practice_statistics
            else:
                combined_df = pd.concat([combined_df, practice_statistics])
            
            logging.info(f'Successfully processed {race_weekend["location"]} {race_weekend["year"]}')
            loop_count += 1
            break  # Break out of main retry loop on success
            
        except Exception as e:
            logging.error(f'Error processing weekend {race_weekend["location"]} {race_weekend["year"]} (Attempt {attempt+1}/{max_retries}): {str(e)}')
            if attempt < max_retries - 1:
                time.sleep(2**attempt)
                continue
            break  # Break after max retries

logging.info('Processing complete, cleaning data...')
logging.info(f'Current shape: {combined_df.shape}')
print(f'\nCurrent shape: {combined_df.shape}')

# Save position history to JSON file
with open('data/position_history.json', 'w') as f:
    json.dump(position_history, f, indent=4)

with pd.option_context('display.max_columns', None):
    display(combined_df)

Processing Melbourne 202544s 2024
Current shape: (958, 680)


,driver_number,fastest_lap_time,fastest_lap_compound,fastest_lap_tyre_age,avg_lap_time,best_s1,best_s2,best_s3,avg_s1,avg_s2,avg_s3,theoretical_best,best_i1_speed,best_i2_speed,avg_i1_speed,avg_i2_speed,total_laps,fastest_lap_SOFT,fastest_lap_tyre_age_SOFT,avg_lap_time_SOFT,best_s1_SOFT,best_s2_SOFT,best_s3_SOFT,theoretical_best_SOFT,avg_s1_SOFT,avg_s2_SOFT,avg_s3_SOFT,laps_SOFT,fastest_lap_MEDIUM,fastest_lap_tyre_age_MEDIUM,avg_lap_time_MEDIUM,best_s1_MEDIUM,best_s2_MEDIUM,best_s3_MEDIUM,theoretical_best_MEDIUM,avg_s1_MEDIUM,avg_s2_MEDIUM,avg_s3_MEDIUM,laps_MEDIUM,fastest_lap_HARD,fastest_lap_tyre_age_HARD,avg_lap_time_HARD,best_s1_HARD,best_s2_HARD,best_s3_HARD,theoretical_best_HARD,avg_s1_HARD,avg_s2_HARD,avg_s3_HARD,laps_HARD,ran_SOFT,ran_HARD,ran_MEDIUM,fastest_lap_time_diff_to_event_median,fastest_lap_time_diff_to_event_min,fastest_lap_time_diff_to_event_max,fastest_lap_time_diff_to_event_mean,fastest_lap_time_pct_diff_to_event_median,fastest_lap_time_pct_diff_to_event_min,fastest_lap_time_pct_diff_to_event_max,fastest_lap_time_pct_diff_to_event_mean,avg_lap_time_diff_to_event_median,avg_lap_time_diff_to_event_min,avg_lap_time_diff_to_event_max,avg_lap_time_diff_to_event_mean,avg_lap_time_pct_diff_to_event_median,avg_lap_time_pct_diff_to_event_min,avg_lap_time_pct_diff_to_event_max,avg_lap_time_pct_diff_to_event_mean,best_s1_diff_to_event_median,best_s1_diff_to_event_min,best_s1_diff_to_event_max,best_s1_diff_to_event_mean,best_s1_pct_diff_to_event_median,best_s1_pct_diff_to_event_min,best_s1_pct_diff_to_event_max,best_s1_pct_diff_to_event_mean,best_s2_diff_to_event_median,best_s2_diff_to_event_min,best_s2_diff_to_event_max,best_s2_diff_to_event_mean,best_s2_pct_diff_to_event_median,best_s2_pct_diff_to_event_min,best_s2_pct_diff_to_event_max,best_s2_pct_diff_to_event_mean,best_s3_diff_to_event_median,best_s3_diff_to_event_min,best_s3_diff_to_event_max,best_s3_diff_to_event_mean,best_s3_pct_diff_to_event_median,best_s3_pct_diff_to_event_min,best_s3_pct_diff_to_event_max,best_s3_pct_diff_to_event_mean,avg_s1_diff_to_event_median,avg_s1_diff_to_event_min,avg_s1_diff_to_event_max,avg_s1_diff_to_event_mean,avg_s1_pct_diff_to_event_median,avg_s1_pct_diff_to_event_min,avg_s1_pct_diff_to_event_max,avg_s1_pct_diff_to_event_mean,avg_s2_diff_to_event_median,avg_s2_diff_to_event_min,avg_s2_diff_to_event_max,avg_s2_diff_to_event_mean,avg_s2_pct_diff_to_event_median,avg_s2_pct_diff_to_event_min,avg_s2_pct_diff_to_event_max,avg_s2_pct_diff_to_event_mean,avg_s3_diff_to_event_median,avg_s3_diff_to_event_min,avg_s3_diff_to_event_max,avg_s3_diff_to_event_mean,avg_s3_pct_diff_to_event_median,avg_s3_pct_diff_to_event_min,avg_s3_pct_diff_to_event_max,avg_s3_pct_diff_to_event_mean,theoretical_best_diff_to_event_median,theoretical_best_diff_to_event_min,theoretical_best_diff_to_event_max,theoretical_best_diff_to_event_mean,theoretical_best_pct_diff_to_event_median,theoretical_best_pct_diff_to_event_min,theoretical_best_pct_diff_to_event_max,theoretical_best_pct_diff_to_event_mean,best_i1_speed_diff_to_event_median,best_i1_speed_diff_to_event_min,best_i1_speed_diff_to_event_max,best_i1_speed_diff_to_event_mean,best_i1_speed_pct_diff_to_event_median,best_i1_speed_pct_diff_to_event_min,best_i1_speed_pct_diff_to_event_max,best_i1_speed_pct_diff_to_event_mean,best_i2_speed_diff_to_event_median,best_i2_speed_diff_to_event_min,best_i2_speed_diff_to_event_max,best_i2_speed_diff_to_event_mean,best_i2_speed_pct_diff_to_event_median,best_i2_speed_pct_diff_to_event_min,best_i2_speed_pct_diff_to_event_max,best_i2_speed_pct_diff_to_event_mean,avg_i1_speed_diff_to_event_median,avg_i1_speed_diff_to_event_min,avg_i1_speed_diff_to_event_max,avg_i1_speed_diff_to_event_mean,avg_i1_speed_pct_diff_to_event_median,avg_i1_speed_pct_diff_to_event_min,avg_i1_speed_pct_diff_to_event_max,avg_i1_speed_pct_diff_to_event_mean,avg_i2_speed_diff_to_event_median,avg_i2_speed_diff_to_event_min,avg_i2_speed_diff_to_event_max,avg_i2_speed_diff_to_event_mean,avg_i2_speed_pct_diff_t

In [4]:
# Clean up the data
combined_df = check_all_ran_values(combined_df)
# Prep for race_df to be included in training data
race_combined_df = combined_df.copy()
# Race position is not needed for training data when predicting quali position
combined_df = combined_df.drop(columns=['race_position'])
combined_df = remove_nan_target_col(combined_df, 'quali_position')
combined_df = fill_nans_with_zero(combined_df)
combined_df = dummy_fastest_lap_compound(combined_df)

logging.info(f'Final dataset shape: {combined_df.shape}')
print(f'New shape: {combined_df.shape}')

with pd.option_context('display.max_columns', None):
    display(combined_df)

New shape: (918, 682)


,driver_number,fastest_lap_time,fastest_lap_tyre_age,avg_lap_time,best_s1,best_s2,best_s3,avg_s1,avg_s2,avg_s3,theoretical_best,best_i1_speed,best_i2_speed,avg_i1_speed,avg_i2_speed,total_laps,fastest_lap_SOFT,fastest_lap_tyre_age_SOFT,avg_lap_time_SOFT,best_s1_SOFT,best_s2_SOFT,best_s3_SOFT,theoretical_best_SOFT,avg_s1_SOFT,avg_s2_SOFT,avg_s3_SOFT,laps_SOFT,fastest_lap_MEDIUM,fastest_lap_tyre_age_MEDIUM,avg_lap_time_MEDIUM,best_s1_MEDIUM,best_s2_MEDIUM,best_s3_MEDIUM,theoretical_best_MEDIUM,avg_s1_MEDIUM,avg_s2_MEDIUM,avg_s3_MEDIUM,laps_MEDIUM,fastest_lap_HARD,fastest_lap_tyre_age_HARD,avg_lap_time_HARD,best_s1_HARD,best_s2_HARD,best_s3_HARD,theoretical_best_HARD,avg_s1_HARD,avg_s2_HARD,avg_s3_HARD,laps_HARD,ran_SOFT,ran_HARD,ran_MEDIUM,fastest_lap_time_diff_to_event_median,fastest_lap_time_diff_to_event_min,fastest_lap_time_diff_to_event_max,fastest_lap_time_diff_to_event_mean,fastest_lap_time_pct_diff_to_event_median,fastest_lap_time_pct_diff_to_event_min,fastest_lap_time_pct_diff_to_event_max,fastest_lap_time_pct_diff_to_event_mean,avg_lap_time_diff_to_event_median,avg_lap_time_diff_to_event_min,avg_lap_time_diff_to_event_max,avg_lap_time_diff_to_event_mean,avg_lap_time_pct_diff_to_event_median,avg_lap_time_pct_diff_to_event_min,avg_lap_time_pct_diff_to_event_max,avg_lap_time_pct_diff_to_event_mean,best_s1_diff_to_event_median,best_s1_diff_to_event_min,best_s1_diff_to_event_max,best_s1_diff_to_event_mean,best_s1_pct_diff_to_event_median,best_s1_pct_diff_to_event_min,best_s1_pct_diff_to_event_max,best_s1_pct_diff_to_event_mean,best_s2_diff_to_event_median,best_s2_diff_to_event_min,best_s2_diff_to_event_max,best_s2_diff_to_event_mean,best_s2_pct_diff_to_event_median,best_s2_pct_diff_to_event_min,best_s2_pct_diff_to_event_max,best_s2_pct_diff_to_event_mean,best_s3_diff_to_event_median,best_s3_diff_to_event_min,best_s3_diff_to_event_max,best_s3_diff_to_event_mean,best_s3_pct_diff_to_event_median,best_s3_pct_diff_to_event_min,best_s3_pct_diff_to_event_max,best_s3_pct_diff_to_event_mean,avg_s1_diff_to_event_median,avg_s1_diff_to_event_min,avg_s1_diff_to_event_max,avg_s1_diff_to_event_mean,avg_s1_pct_diff_to_event_median,avg_s1_pct_diff_to_event_min,avg_s1_pct_diff_to_event_max,avg_s1_pct_diff_to_event_mean,avg_s2_diff_to_event_median,avg_s2_diff_to_event_min,avg_s2_diff_to_event_max,avg_s2_diff_to_event_mean,avg_s2_pct_diff_to_event_median,avg_s2_pct_diff_to_event_min,avg_s2_pct_diff_to_event_max,avg_s2_pct_diff_to_event_mean,avg_s3_diff_to_event_median,avg_s3_diff_to_event_min,avg_s3_diff_to_event_max,avg_s3_diff_to_event_mean,avg_s3_pct_diff_to_event_median,avg_s3_pct_diff_to_event_min,avg_s3_pct_diff_to_event_max,avg_s3_pct_diff_to_event_mean,theoretical_best_diff_to_event_median,theoretical_best_diff_to_event_min,theoretical_best_diff_to_event_max,theoretical_best_diff_to_event_mean,theoretical_best_pct_diff_to_event_median,theoretical_best_pct_diff_to_event_min,theoretical_best_pct_diff_to_event_max,theoretical_best_pct_diff_to_event_mean,best_i1_speed_diff_to_event_median,best_i1_speed_diff_to_event_min,best_i1_speed_diff_to_event_max,best_i1_speed_diff_to_event_mean,best_i1_speed_pct_diff_to_event_median,best_i1_speed_pct_diff_to_event_min,best_i1_speed_pct_diff_to_event_max,best_i1_speed_pct_diff_to_event_mean,best_i2_speed_diff_to_event_median,best_i2_speed_diff_to_event_min,best_i2_speed_diff_to_event_max,best_i2_speed_diff_to_event_mean,best_i2_speed_pct_diff_to_event_median,best_i2_speed_pct_diff_to_event_min,best_i2_speed_pct_diff_to_event_max,best_i2_speed_pct_diff_to_event_mean,avg_i1_speed_diff_to_event_median,avg_i1_speed_diff_to_event_min,avg_i1_speed_diff_to_event_max,avg_i1_speed_diff_to_event_mean,avg_i1_speed_pct_diff_to_event_median,avg_i1_speed_pct_diff_to_event_min,avg_i1_speed_pct_diff_to_event_max,avg_i1_speed_pct_diff_to_event_mean,avg_i2_speed_diff_to_event_median,avg_i2_speed_diff_to_event_min,avg_i2_speed_diff_to_event_max,avg_i2_speed_diff_to_event_mean,avg_i2_speed_pct_diff_to_event_median,avg_i2

In [5]:
ran_columns = [col for col in combined_df.columns if 'ran_' in col]
target_col = 'quali_position'
other_cols = [col for col in combined_df.columns if col not in ran_columns and col != target_col]

combined_df = combined_df[other_cols + ran_columns + [target_col]]

with pd.option_context('display.max_columns', None):
    display(combined_df)

,driver_number,fastest_lap_time,fastest_lap_tyre_age,avg_lap_time,best_s1,best_s2,best_s3,avg_s1,avg_s2,avg_s3,theoretical_best,best_i1_speed,best_i2_speed,avg_i1_speed,avg_i2_speed,total_laps,fastest_lap_SOFT,fastest_lap_tyre_age_SOFT,avg_lap_time_SOFT,best_s1_SOFT,best_s2_SOFT,best_s3_SOFT,theoretical_best_SOFT,avg_s1_SOFT,avg_s2_SOFT,avg_s3_SOFT,laps_SOFT,fastest_lap_MEDIUM,fastest_lap_tyre_age_MEDIUM,avg_lap_time_MEDIUM,best_s1_MEDIUM,best_s2_MEDIUM,best_s3_MEDIUM,theoretical_best_MEDIUM,avg_s1_MEDIUM,avg_s2_MEDIUM,avg_s3_MEDIUM,laps_MEDIUM,fastest_lap_HARD,fastest_lap_tyre_age_HARD,avg_lap_time_HARD,best_s1_HARD,best_s2_HARD,best_s3_HARD,theoretical_best_HARD,avg_s1_HARD,avg_s2_HARD,avg_s3_HARD,laps_HARD,fastest_lap_time_diff_to_event_median,fastest_lap_time_diff_to_event_min,fastest_lap_time_diff_to_event_max,fastest_lap_time_diff_to_event_mean,fastest_lap_time_pct_diff_to_event_median,fastest_lap_time_pct_diff_to_event_min,fastest_lap_time_pct_diff_to_event_max,fastest_lap_time_pct_diff_to_event_mean,avg_lap_time_diff_to_event_median,avg_lap_time_diff_to_event_min,avg_lap_time_diff_to_event_max,avg_lap_time_diff_to_event_mean,avg_lap_time_pct_diff_to_event_median,avg_lap_time_pct_diff_to_event_min,avg_lap_time_pct_diff_to_event_max,avg_lap_time_pct_diff_to_event_mean,best_s1_diff_to_event_median,best_s1_diff_to_event_min,best_s1_diff_to_event_max,best_s1_diff_to_event_mean,best_s1_pct_diff_to_event_median,best_s1_pct_diff_to_event_min,best_s1_pct_diff_to_event_max,best_s1_pct_diff_to_event_mean,best_s2_diff_to_event_median,best_s2_diff_to_event_min,best_s2_diff_to_event_max,best_s2_diff_to_event_mean,best_s2_pct_diff_to_event_median,best_s2_pct_diff_to_event_min,best_s2_pct_diff_to_event_max,best_s2_pct_diff_to_event_mean,best_s3_diff_to_event_median,best_s3_diff_to_event_min,best_s3_diff_to_event_max,best_s3_diff_to_event_mean,best_s3_pct_diff_to_event_median,best_s3_pct_diff_to_event_min,best_s3_pct_diff_to_event_max,best_s3_pct_diff_to_event_mean,avg_s1_diff_to_event_median,avg_s1_diff_to_event_min,avg_s1_diff_to_event_max,avg_s1_diff_to_event_mean,avg_s1_pct_diff_to_event_median,avg_s1_pct_diff_to_event_min,avg_s1_pct_diff_to_event_max,avg_s1_pct_diff_to_event_mean,avg_s2_diff_to_event_median,avg_s2_diff_to_event_min,avg_s2_diff_to_event_max,avg_s2_diff_to_event_mean,avg_s2_pct_diff_to_event_median,avg_s2_pct_diff_to_event_min,avg_s2_pct_diff_to_event_max,avg_s2_pct_diff_to_event_mean,avg_s3_diff_to_event_median,avg_s3_diff_to_event_min,avg_s3_diff_to_event_max,avg_s3_diff_to_event_mean,avg_s3_pct_diff_to_event_median,avg_s3_pct_diff_to_event_min,avg_s3_pct_diff_to_event_max,avg_s3_pct_diff_to_event_mean,theoretical_best_diff_to_event_median,theoretical_best_diff_to_event_min,theoretical_best_diff_to_event_max,theoretical_best_diff_to_event_mean,theoretical_best_pct_diff_to_event_median,theoretical_best_pct_diff_to_event_min,theoretical_best_pct_diff_to_event_max,theoretical_best_pct_diff_to_event_mean,best_i1_speed_diff_to_event_median,best_i1_speed_diff_to_event_min,best_i1_speed_diff_to_event_max,best_i1_speed_diff_to_event_mean,best_i1_speed_pct_diff_to_event_median,best_i1_speed_pct_diff_to_event_min,best_i1_speed_pct_diff_to_event_max,best_i1_speed_pct_diff_to_event_mean,best_i2_speed_diff_to_event_median,best_i2_speed_diff_to_event_min,best_i2_speed_diff_to_event_max,best_i2_speed_diff_to_event_mean,best_i2_speed_pct_diff_to_event_median,best_i2_speed_pct_diff_to_event_min,best_i2_speed_pct_diff_to_event_max,best_i2_speed_pct_diff_to_event_mean,avg_i1_speed_diff_to_event_median,avg_i1_speed_diff_to_event_min,avg_i1_speed_diff_to_event_max,avg_i1_speed_diff_to_event_mean,avg_i1_speed_pct_diff_to_event_median,avg_i1_speed_pct_diff_to_event_min,avg_i1_speed_pct_diff_to_event_max,avg_i1_speed_pct_diff_to_event_mean,avg_i2_speed_diff_to_event_median,avg_i2_speed_diff_to_event_min,avg_i2_speed_diff_to_event_max,avg_i2_speed_diff_to_event_mean,avg_i2_speed_pct_diff_to_event_median,avg_i2_speed_pct_diff_to_event_min,

In [6]:
# Create data directory if it doesn't exist
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Save the combined dataframe to a CSV file
combined_df.to_csv(os.path.join(data_dir, 'qualifying_data.csv'), index=False)

# Create json of all training data columns in array
quali_data_columns = combined_df.columns.tolist()
with open('data/qualifying_data_columns.json', 'w') as f:
    json.dump(quali_data_columns, f, indent=4)

## For race_df

In [7]:
race_combined_df = remove_nan_target_col(race_combined_df, 'quali_position')
race_combined_df = remove_nan_target_col(race_combined_df, 'race_position')
race_combined_df = fill_nans_with_zero(race_combined_df)
race_combined_df = dummy_fastest_lap_compound(race_combined_df)

ran_columns = [col for col in race_combined_df.columns if 'ran_' in col]
target_cols = ['quali_position', 'race_position']
other_cols = [col for col in race_combined_df.columns if col not in ran_columns and col not in target_cols]

race_combined_df = race_combined_df[other_cols + ran_columns + target_cols]

with pd.option_context('display.max_columns', None):
    display(race_combined_df)

race_combined_df.to_csv(os.path.join(data_dir, 'race_data.csv'), index=False)

race_data_columns = race_combined_df.columns.tolist()
with open('data/race_data_columns.json', 'w') as f:
    json.dump(race_data_columns, f, indent=4)

,driver_number,fastest_lap_time,fastest_lap_tyre_age,avg_lap_time,best_s1,best_s2,best_s3,avg_s1,avg_s2,avg_s3,theoretical_best,best_i1_speed,best_i2_speed,avg_i1_speed,avg_i2_speed,total_laps,fastest_lap_SOFT,fastest_lap_tyre_age_SOFT,avg_lap_time_SOFT,best_s1_SOFT,best_s2_SOFT,best_s3_SOFT,theoretical_best_SOFT,avg_s1_SOFT,avg_s2_SOFT,avg_s3_SOFT,laps_SOFT,fastest_lap_MEDIUM,fastest_lap_tyre_age_MEDIUM,avg_lap_time_MEDIUM,best_s1_MEDIUM,best_s2_MEDIUM,best_s3_MEDIUM,theoretical_best_MEDIUM,avg_s1_MEDIUM,avg_s2_MEDIUM,avg_s3_MEDIUM,laps_MEDIUM,fastest_lap_HARD,fastest_lap_tyre_age_HARD,avg_lap_time_HARD,best_s1_HARD,best_s2_HARD,best_s3_HARD,theoretical_best_HARD,avg_s1_HARD,avg_s2_HARD,avg_s3_HARD,laps_HARD,fastest_lap_time_diff_to_event_median,fastest_lap_time_diff_to_event_min,fastest_lap_time_diff_to_event_max,fastest_lap_time_diff_to_event_mean,fastest_lap_time_pct_diff_to_event_median,fastest_lap_time_pct_diff_to_event_min,fastest_lap_time_pct_diff_to_event_max,fastest_lap_time_pct_diff_to_event_mean,avg_lap_time_diff_to_event_median,avg_lap_time_diff_to_event_min,avg_lap_time_diff_to_event_max,avg_lap_time_diff_to_event_mean,avg_lap_time_pct_diff_to_event_median,avg_lap_time_pct_diff_to_event_min,avg_lap_time_pct_diff_to_event_max,avg_lap_time_pct_diff_to_event_mean,best_s1_diff_to_event_median,best_s1_diff_to_event_min,best_s1_diff_to_event_max,best_s1_diff_to_event_mean,best_s1_pct_diff_to_event_median,best_s1_pct_diff_to_event_min,best_s1_pct_diff_to_event_max,best_s1_pct_diff_to_event_mean,best_s2_diff_to_event_median,best_s2_diff_to_event_min,best_s2_diff_to_event_max,best_s2_diff_to_event_mean,best_s2_pct_diff_to_event_median,best_s2_pct_diff_to_event_min,best_s2_pct_diff_to_event_max,best_s2_pct_diff_to_event_mean,best_s3_diff_to_event_median,best_s3_diff_to_event_min,best_s3_diff_to_event_max,best_s3_diff_to_event_mean,best_s3_pct_diff_to_event_median,best_s3_pct_diff_to_event_min,best_s3_pct_diff_to_event_max,best_s3_pct_diff_to_event_mean,avg_s1_diff_to_event_median,avg_s1_diff_to_event_min,avg_s1_diff_to_event_max,avg_s1_diff_to_event_mean,avg_s1_pct_diff_to_event_median,avg_s1_pct_diff_to_event_min,avg_s1_pct_diff_to_event_max,avg_s1_pct_diff_to_event_mean,avg_s2_diff_to_event_median,avg_s2_diff_to_event_min,avg_s2_diff_to_event_max,avg_s2_diff_to_event_mean,avg_s2_pct_diff_to_event_median,avg_s2_pct_diff_to_event_min,avg_s2_pct_diff_to_event_max,avg_s2_pct_diff_to_event_mean,avg_s3_diff_to_event_median,avg_s3_diff_to_event_min,avg_s3_diff_to_event_max,avg_s3_diff_to_event_mean,avg_s3_pct_diff_to_event_median,avg_s3_pct_diff_to_event_min,avg_s3_pct_diff_to_event_max,avg_s3_pct_diff_to_event_mean,theoretical_best_diff_to_event_median,theoretical_best_diff_to_event_min,theoretical_best_diff_to_event_max,theoretical_best_diff_to_event_mean,theoretical_best_pct_diff_to_event_median,theoretical_best_pct_diff_to_event_min,theoretical_best_pct_diff_to_event_max,theoretical_best_pct_diff_to_event_mean,best_i1_speed_diff_to_event_median,best_i1_speed_diff_to_event_min,best_i1_speed_diff_to_event_max,best_i1_speed_diff_to_event_mean,best_i1_speed_pct_diff_to_event_median,best_i1_speed_pct_diff_to_event_min,best_i1_speed_pct_diff_to_event_max,best_i1_speed_pct_diff_to_event_mean,best_i2_speed_diff_to_event_median,best_i2_speed_diff_to_event_min,best_i2_speed_diff_to_event_max,best_i2_speed_diff_to_event_mean,best_i2_speed_pct_diff_to_event_median,best_i2_speed_pct_diff_to_event_min,best_i2_speed_pct_diff_to_event_max,best_i2_speed_pct_diff_to_event_mean,avg_i1_speed_diff_to_event_median,avg_i1_speed_diff_to_event_min,avg_i1_speed_diff_to_event_max,avg_i1_speed_diff_to_event_mean,avg_i1_speed_pct_diff_to_event_median,avg_i1_speed_pct_diff_to_event_min,avg_i1_speed_pct_diff_to_event_max,avg_i1_speed_pct_diff_to_event_mean,avg_i2_speed_diff_to_event_median,avg_i2_speed_diff_to_event_min,avg_i2_speed_diff_to_event_max,avg_i2_speed_diff_to_event_mean,avg_i2_speed_pct_diff_to_event_median,avg_i2_speed_pct_diff_to_event_min,